In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
# Install OpenNMT-py 2.x
# NOTE: By the end of the insatallation, it might ask for restarting the runtime...
# In this case, just click the "RESTART RUNTIME" button.

!pip3 install OpenNMT-py
!pip3 install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.6/219.6 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 97.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [5]:
import torch

torch.cuda.is_available()
torch.cuda.get_device_name(0)
folder = "drive/MyDrive/Colab Notebooks/seqs/names-filtered-no-wasm-ty/seq2"

In [6]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
!rm -rf sample
!rm -rf vocab
!rm model*
!rm pred*

config = '''
save_data: samples
src_vocab: 'vocab/src'
tgt_vocab: 'vocab/tgt'
overwrite: False

data:
    corpus_1:
        path_src: './{}/x_train.txt'
        path_tgt: './{}/y_train.txt'
        transforms: [onmt_tokenize]
        weight: 1
    valid:
        path_src: './{}/x_dev.txt'
        path_tgt: './{}/y_dev.txt'
        transforms: [onmt_tokenize]

world_size: 1

gpu_ranks: [0]

save_model: model
save_checkpoint_steps: 1000

valid_steps: 1000

train_steps: 50000
log_file: 'train.log'
report_every: 50 

early_stopping: 3

optim: 'adam'
learning_rate: 0.001 

dropout: 0.2

batch_size: 128

word_vec_size: 100

encoder_type: 'brnn'

rnn_size: 512

dec_layers: 1

src_seq_length_trunc: 500
src_seq_length: 10000
tgt_seq_length_trunc: 500
tgt_seq_length: 1000
'''.format(folder,folder,folder,folder)

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

!cat config.yaml

rm: cannot remove 'model*': No such file or directory
rm: cannot remove 'pred*': No such file or directory

save_data: samples
src_vocab: 'vocab/src'
tgt_vocab: 'vocab/tgt'
overwrite: False

data:
    corpus_1:
        path_src: './drive/MyDrive/Colab Notebooks/seqs/names-filtered-no-wasm-ty/seq2/x_train.txt'
        path_tgt: './drive/MyDrive/Colab Notebooks/seqs/names-filtered-no-wasm-ty/seq2/y_train.txt'
        transforms: [onmt_tokenize]
        weight: 1
    valid:
        path_src: './drive/MyDrive/Colab Notebooks/seqs/names-filtered-no-wasm-ty/seq2/x_dev.txt'
        path_tgt: './drive/MyDrive/Colab Notebooks/seqs/names-filtered-no-wasm-ty/seq2/y_dev.txt'
        transforms: [onmt_tokenize]

world_size: 1

gpu_ranks: [0]

save_model: model
save_checkpoint_steps: 1000

valid_steps: 1000

train_steps: 50000
log_file: 'train.log'
report_every: 50 

early_stopping: 3

optim: 'adam'
learning_rate: 0.001 

dropout: 0.2

batch_size: 128

word_vec_size: 100

encoder_type: 'brnn'

rnn_s

In [7]:
# Build Vocabulary
!onmt_build_vocab --config config.yaml --n_sample=100 --dump_samples
!onmt_build_vocab --config config.yaml --n_sample=-1 --num_threads=10 --overwrite

[2023-03-18 15:19:54,029 INFO] Parsed pyonmttok kwargs for src: {'mode': 'none'}
[2023-03-18 15:19:54,029 INFO] Parsed pyonmttok kwargs for tgt: {'mode': 'none'}
[2023-03-18 15:19:54,029 WARNING] No subword method will be applied.
[2023-03-18 15:19:54,029 WARNING] No subword method will be applied.
[2023-03-18 15:19:54,029 WARNING] No subword method will be applied.
[2023-03-18 15:19:54,029 INFO] Counter vocab from 100 samples.
[2023-03-18 15:19:54,029 INFO] Build vocab on 100 transformed examples/corpus.
[2023-03-18 15:19:54,029 INFO] The samples on which the vocab is built will be dumped to disk. It may slow down the process.
[2023-03-18 15:19:54,041 INFO] Parsed pyonmttok kwargs for src: {'mode': 'none'}
[2023-03-18 15:19:54,041 INFO] Parsed pyonmttok kwargs for tgt: {'mode': 'none'}
[2023-03-18 15:19:54,041 WARNING] No subword method will be applied.
[2023-03-18 15:19:54,042 WARNING] No subword method will be applied.
[2023-03-18 15:19:54,042 WARNING] No subword method will be appl

In [8]:
# Check if GPU is active
# If not, go to "Runtime" menu > "Change runtime type" > "GPU"

!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-80de0e34-d89e-3cd6-2cd6-2296a962cba2)


In [9]:
# Make sure the GPU is visable to PyTorch

import torch

gpu_id = torch.cuda.current_device()
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(gpu_id))

True
Tesla T4


In [10]:
# Train the NMT model

!onmt_train -config config.yaml

[2023-03-18 15:20:06,459 INFO] Parsed 2 corpora from -data.
[2023-03-18 15:20:06,459 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-03-18 15:20:06,587 INFO] Building model...
[2023-03-18 15:20:14,307 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(32768, 100, padding_idx=1)
        )
      )
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (rnn): LSTM(100, 250, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(176, 100, padding_idx=1)
        )
      )
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.2, inplace=False)
      (layers): ModuleList(
        (0): LSTMCe

In [11]:
!apt-get install ripgrep

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  ripgrep
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 1,173 kB of archives.
After this operation, 4,293 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 ripgrep amd64 11.0.2-1ubuntu0.1 [1,173 kB]
Fetched 1,173 kB in 2s (552 kB/s)
Selecting previously unselected package ripgrep.
(Reading database ... 128276 files and directories currently installed.)
Preparing to unpack .../ripgrep_11.0.2-1ubuntu0.1_amd64.deb ...
Unpacking ripgrep (11.0.2-1ubuntu0.1) ...
Setting up ripgrep (11.0.2-1ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...


In [12]:
!rm model_best.pt
!ln -s "model_step_$(rg '.* Best model found at step (\d+)' train.log -r '$1' | tail -n1).pt" model_best.pt
!ls -lt

rm: cannot remove 'model_best.pt': No such file or directory
total 604228
lrwxrwxrwx 1 root root        18 Mar 18 15:35 model_best.pt -> model_step_2000.pt
-rw-r--r-- 1 root root 103109682 Mar 18 15:35 model_step_6000.pt
-rw-r--r-- 1 root root     27261 Mar 18 15:35 train.log
-rw-r--r-- 1 root root 103109682 Mar 18 15:33 model_step_5000.pt
-rw-r--r-- 1 root root 103109682 Mar 18 15:30 model_step_4000.pt
-rw-r--r-- 1 root root 103109682 Mar 18 15:28 model_step_3000.pt
-rw-r--r-- 1 root root 103109682 Mar 18 15:25 model_step_2000.pt
-rw-r--r-- 1 root root 103109682 Mar 18 15:23 model_step_1000.pt
drwxr-xr-x 2 root root      4096 Mar 18 15:19 vocab
drwxr-xr-x 2 root root      4096 Mar 18 15:19 sample
-rw-r--r-- 1 root root      1013 Mar 18 15:19 config.yaml
drwx------ 5 root root      4096 Mar 18 15:19 drive
drwxr-xr-x 1 root root      4096 Mar 16 13:45 sample_data


In [13]:
sh = """
onmt_translate --src='{}/x_test.txt' --model='./model_best.pt' --output="{}/y_test.acc.txt" --log_file='predict.log' --n_best=5 --beam_size=5 --report_time --gpu=0 --batch_size=100
python3 drive/MyDrive/Colab\ Notebooks/eval.py --log="eval.log" --predictions="{}/y_test.acc.txt" --ground-truth="{}/y_test.txt"
""".format(folder, folder, folder, folder)
with open('script.sh', 'w+') as file:
  file.write(sh)

!bash script.sh

[2023-03-18 15:38:45,007 INFO] PRED SCORE: -0.1772, PRED PPL: 1.19 NB SENTENCES: 43168
[2023-03-18 15:38:45,008 INFO] Total translation time (s): 169.9
[2023-03-18 15:38:45,008 INFO] Average translation time (ms): 3.9
[2023-03-18 15:38:45,008 INFO] Tokens per second: 514.9
2023-03-18 15:38:48 INFO     reading predictions from 'drive/MyDrive/Colab Notebooks/seqs/names-filtered-no-wasm-ty/seq2/y_test.acc.txt'...
2023-03-18 15:38:49 INFO     predictions:              215840
2023-03-18 15:38:49 INFO     ground truth samples:      43168
2023-03-18 15:38:49 INFO     top-k: 5
2023-03-18 15:38:49 INFO     perfect match accuracy:
2023-03-18 15:38:49 INFO       top-1: 44.42% (19174/43168)
2023-03-18 15:38:49 INFO       top-2: 59.85% (25838/43168)
2023-03-18 15:38:49 INFO       top-3: 67.57% (29169/43168)
2023-03-18 15:38:49 INFO       top-4: 70.89% (30601/43168)
2023-03-18 15:38:49 INFO       top-5: 73.98% (31937/43168)
2023-03-18 15:38:49 INFO     average type prefix score (top-1): 0.544
2023-0